# Praca domowa 1 - warsztaty badawcze

## Autorzy: Jakub Kozieł, Jan Skwarek, Tomasz Nocoń

# Wstęp

In [ ]:
! pip install swifter
! pip install pandas
! pip install textacy
! pip install spacy

In [ ]:
! python -m spacy download en_core_web_sm

In [ ]:
import spacy
import pandas as pd
from tqdm.auto import tqdm 
import swifter
import plotly.express as px
from wordcloud import WordCloud
from matplotlib import pyplot as plt
import textacy
from collections import Counter
import random

pd.options.plotting.backend = "plotly"
random.seed(123)

# Preprocessing

As previously we work on data from https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/6MZN76

In [ ]:
!wget -O data.tar.gz https://dataverse.harvard.edu/api/access/datafile/:persistentId?persistentId=doi:10.7910/DVN/6MZN76/CRUNF0

In [ ]:
!tar -xf data.tar.gz

In [ ]:
en = spacy.load("en_core_web_sm") #so far we keep on execution time


In [ ]:
df = pd.read_table('Dail_debates_1919-2013.tab')

In [ ]:
df.date = pd.to_datetime(df.date)

Filtrujemy ramkę danych tak, żeby pokazywała lata 2007-2010. Wtedy na świecie trwał kryzys ekonomiczny. Być może znajdziemy jakieś ciekawe dane pod tym kątem.

In [ ]:
#we are going to use subsample from 2007 to 2010, being interested in the years before and after economical crisis
df = df[(df.date.dt.year>=2007)&(df.date.dt.year<=2010)]

In [ ]:
df.info()

431 tys. wierszy to jednak nadal zbyt dużo. Wybierzemy z tego podzbiór, zachowując podobną ilość obserwacji z każdego roku

In [ ]:
df["year"] = df.date.dt.year

df = df.groupby('year', group_keys=False).apply(lambda x: x.sample(3500, random_state=123))

df.drop("year", axis = 1)

In [ ]:
df.columns

In [ ]:
df["speech_en"] = df['speech'].swifter.apply(en)

Czas na powyższą operację <7min

# Zależności

In [ ]:
df.party_name.unique()

In [ ]:
df.nunique()

W analizowanych przez nas latach kolicję tworzyły partie FF-Green-PD

Naszym pomysłem na ten zbiór było zbadanie czy kryzys światowy z roku 2008 miał wpył na temtyke wypowiedzi w rządzie. W tym celu wzieliśmy okres 2007-2010, żeby zobaczyć czy faktycznie kryzys mial w latach 2008-2009 mial wpływ na to co było mówione

Skupiliśmy sie na charakterze wypowiedzi wewnątrz partii.

In [ ]:
df.party_name.unique()

In [ ]:
year = 2009
df_year = df[df.date.dt.year == year]
parties = df_year['party_name'].unique()
for party in parties:
    temp = df_year[df_year['party_name'] == party]
    lemmas = temp.speech_en.apply(lambda doc: [token.lemma_ for token in doc if not token.is_stop if not token.is_punct])
    word_counts = dict(Counter(lemmas.sum()).most_common(30))
    wc = WordCloud(width=800, height=400)
    wc.generate_from_frequencies(frequencies=word_counts)
    plt.figure(figsize=(10,8))
    plt.title(party+str(year))
    plt.imshow(wc)

To co tutaj zrobiliśmy to podzieliliśmy wypowiedzi na osoby z partii. Dodotkowo manipulowalismy sobie latami po to, żeby zobaczyć czy faktycznie okres kryzysu spowoduje jakieś zmiany w kluczowych temtatch. Z przeprowadzonych eksperymentow wynika, że pojawia się wzmianki na ten temat, np lata 2008 czy 2009 sa cześciej pojawiaja się niż te pozstałe, jednak nie ma jakis oczywistych wniosków. To co możemy powiedziec, ze słowo Minister jest jednym z bardziej popularych słów, nie jest jednak to prawda dla partii socjalnych, np: Workers Party gdzie używaja oni irlandziekgo określenia na to stanowisko. To co mozemy powiedzieć, to ze mimo wszytko jednak teamty skupiaja sie na sprawch ważnych dla pogladów (zieloni mówiąo energi) partii czy dzijacych sie w kraju ( healt care czy school). Ciekawe może być to, ze np w 2009 bardzo częstym słowem w wypowiedziach partii Progressive Democtrats było Art czy np wybieganie w przysłość do roku 2012.

Teraz podobna analiza tylko z wywołaniem funckji noun_chunks, czyli chcemy zobaczyć jakie grupy słów czesto się ze sobą pojawiały

In [ ]:
year = 2008
df_year = df[df.date.dt.year == year]
parties = df_year['party_name'].unique()
for party in parties:
    temp = df_year[df_year['party_name'] == party]
    lemmas = temp.speech_en.apply(lambda doc: list(doc.noun_chunks))
    lemmas = lemmas.apply(lambda x: [''.join(str(el)) for el in x if len(el) > 2])
    word_counts = dict(Counter(lemmas.sum()).most_common(30))
    wc = WordCloud(width=800, height=400)
    wc.generate_from_frequencies(frequencies=word_counts)
    plt.figure(figsize=(10,8))
    plt.title(party+str(year))
    plt.imshow(wc)

Faktycznie zestawienie ze sobą większej grupy słów daje ciekwsze rezulataty. Co wiecej lepiej się rozumie kontekst tych fraz oraz jest łatwiejsza ich interpretowalonść. (Przyczyny występowania danych słów ze sobą)

# Praca z Dail_debates_1937-2011_ministers.tab

## Wstępne informacje

In [ ]:
df_ministers = pd.read_table('Dail_debates_1937-2011_ministers.tab')

In [ ]:
df_ministers.head()

Widać, że będą w tej ramce ministrowie pełniący wiele funkcji, w różnych departamentach i z różnymi długościami kadencji. Trzeba o tym będzie pamiętać i uwzględniać to w analizie.

In [ ]:
df_ministers.info()

Widać, że w ramce będą potencjalne nulle, głównie w dacie zkończenia kadencji, co mogło nie nastąpić przed rokiem 2011. Ograniczmy się do naszych lat i sprawdźmy, to ponownie.

In [ ]:
df_ministers.start_date = pd.to_datetime(df_ministers.start_date)
df_ministers.end_date = pd.to_datetime(df_ministers.end_date)

In [ ]:
df_ministers = df_ministers[( (df_ministers.start_date.dt.year<=2010) & (df_ministers.start_date.dt.year>=2007) )|( (df_ministers.end_date.dt.year<=2010) & (df_ministers.end_date.dt.year>=2007) )]

In [ ]:
df_ministers.info()

Jak się okazuje problem z nullami znika sam. Pozbądźmy się jeszcze kilku kolumn dla nas zbytecznych.

In [ ]:
df_ministers = df_ministers.drop(["start_day", "start_month", "start_year", "end_day", "end_month", "end_year", "name"], axis = 1)

In [ ]:
df_ministers.head()

Utwórzmy kilka przydatnych zbiorków. Set z indeksami ministrów w interesujących nas latach oraz 2 słowniki, które będą wiązać pozycje lub departament z danym member ID.

In [ ]:
ministers_ids = set(df_ministers.memberID.values)
#ministers_ids

In [ ]:
dict_pos = df_ministers.groupby('position')['memberID'].apply(set).to_dict()
#dict_pos

In [ ]:
dict_depart = df_ministers.groupby('department')['memberID'].apply(set).to_dict()
#dict_depart

In [ ]:
len(ministers_ids)

Zatem w tym okresie było 52 osobnych ministrów róznego typu, w tym premierzy.

In [ ]:
print(dict_depart.keys())
print(len(dict_depart.keys()))

Powyżej nazwy 27 resortów, zawsze można sobie wypritnować ten słownik, aby dodatkowo zyskać intuicję co do potencjalnej ilości ministrów w danym obszarze w latach 2007-2010 włącznie.

In [ ]:
print(dict_pos.keys())
print(len(dict_pos.keys()))

Budzić zainteresowanie mogą 2 obco brzmiące nazwy. Taoiseach to osoba premiera w Irlandzkim rządzie. W jego zakresie jest m.in. mianowanie osoby na urząd Tánaiste, czyli wicepremiera.

In [ ]:
print("Liczba premierów:", len(dict_pos["Taoiseach"]), "Liczba wicepremierów:", len(dict_pos["Tánaiste"]))

Większa liczba wicepremirów mówi, że jeden z nich został odwołany lub zrezygnował z pełnienia funkcji. Dokonując analizy osobno dla tych, tak mało licznych grup, trzeba pamiętać, że na charakter wypowiedzi wpływ będzie mieć zarówno pełniona funkcja, ale co ważne, sposób wypowiadania się danej jednostki. (Dłuższe/krótsze wypowiedzi mogą nie być powiązane z funkcją premiera, a jedynie sposobem wygłaszania przemówień i prowadzenia polityki tego z obecnej kadencji.)

In [ ]:
df_ministers.nunique()

Luźna myśl: w 2008 roku było także referendum na temat retyfikacji traktatu lizbońskiego, pytanie czy mogło być to w jakikolwiek sposób odzwierciedlone w naszym zbiorku?? Ostatecznie ludzie opowiedzieli się przeciwko, co stoi w opozycji do wczesnych sądaży.

Zastanawiać może, skąd taka liczba urzędów, 3 przypadające na okres 2007-2010. Spodziewalibyśmy się 2: jeden przed wyborami w 2007 roku, drugi po nich. To może również tłumaczyć 3 wicepremirów wcześniej i przeczyć wnioskowi, że któryś został odwołany. Odpowiedzi dostarcza wikipedia. "There were two Governments of the 30th Dáil, which was elected at the 2007 general election on 24 May 2007. The 27th Government of Ireland (14 June 2007 – 7 May 2008) was led by Bertie Ahern as Taoiseach, and the 28th Government of Ireland (7 May 2008 – 9 March 2011) was led by Brian Cowen as Taoiseach." W obrędbie tej samej koalicji 2 partii nastąpiła wymiana rządu. Premierem nowego stał się Brian Cowen, uprzednio będący na funkcji wicepremiera. (Fakt wystąpowania tylko 2 premierów tłumaczy fakt, że Bertie Ahern pełnił tę funkcję także przed rokiem 2007, bo od 1997).

## Długość tekstów ministrowie vs. nieministrowie

In [ ]:
#non-ministers
doc_lens_nm = df.loc[~df['memberID'].isin(ministers_ids)].speech_en.str.len()
doc_lens_nm.hist(title = "Nieministrowie",log_y=True)

In [ ]:
#ministers

doc_lens_m = df.loc[df['memberID'].isin(ministers_ids)].speech_en.str.len()
doc_lens_m.hist(title = "Ministrowie", log_y=True)

Można wyciągnąć wstępne wnioski co do długości speechy i rozkładów w poszczególnych grupach. Zastanawiający jest potencjalny brak dużej różnicy w ogólnej liczbie wystąpień. Sprawdźmy zatem ile jest nieministrów w kontrze do 52 ministrów.

In [ ]:
print("Liczba speechy nieministrów:", len(df.loc[~df['memberID'].isin(ministers_ids)]), "liczba speechy ministrów:", len(df.loc[df['memberID'].isin(ministers_ids)]))

In [ ]:
print("Liczba wypowiadających się nieministrów:", len(set(df.loc[~df['memberID'].isin(ministers_ids)]['memberID'])))

Dosyć zaskakująca dysproporcja, wielu wypowiadających się pełniło wcześniej lub poźniej funkcję ministra. Można to jednak uzasadniaż aż 3 rządami oraz faktem, że większość ze 160 miejsc będzie mieć rządząca koalicja. 
Istotniejsze jest, że można by rozważyć pomysł dokonywania analiz ze względu także czy dana wypowiedź padła z okresu pełnienia funkcji ministra.

In [ ]:
data = [doc_lens_m, doc_lens_nm]

# Multiple box plots on one Axes
fig, ax = plt.subplots(figsize=(19, 13))
#fig = plt.figure(figsize=(19, 3))
ax.boxplot(data)
#ax.set_xticklabels["Ministers", "Non-ministers"]
plt.xticks([1, 2], ["Ministers", "Non-ministers"])
ax.set_yscale('log')
plt.show()

Ministrowie mają średnio dłuższe przemówienia, przesunięte są także odpowiednie kwantyle w kierunku większych wartości. Różnica mogłaby być jeszcze większa biorąc np. długość wypowiedzi tylko z okresu pełnienia funkcji.

## Word clouds ministrowie vs. nieministrowie

In [ ]:
def cloud_from_lemmas(lemmas):
  word_counts = Counter(lemmas.sum())

  wc = WordCloud(width=800, height=400)
  wc.generate_from_frequencies(frequencies=word_counts)
  plt.figure(figsize=(10,8))
  plt.imshow(wc)

Słowa dla nieministrów.

In [ ]:
lemmas_nm = df.loc[~df['memberID'].isin(ministers_ids)].speech_en.apply(lambda doc: [token.lemma_ for token in doc if not token.is_stop if not token.is_punct])

cloud_from_lemmas(lemmas_nm)

Słowa dla ministrów:

In [ ]:
lemmas_m = df.loc[~df['memberID'].isin(ministers_ids)].speech_en.apply(lambda doc: [token.lemma_ for token in doc if not token.is_stop if not token.is_punct])

cloud_from_lemmas(lemmas_m)

Mimo faktu, że duże przecięcie będzie wynikać z faktu występowania słów charakterystycznych dla polityki, zacznijmy rozważać speeche ministrów jedynie za ich kadencji. 

Odnotujmy już teraz fakt, że analizując słowa minstrów z tych lat, analizujemy jednocześnie słowa członków koalicji rządzącej FF-Green-PD. Słowa powiązane z wartościami partii, a nie tylko pozycją.

In [ ]:
min_years_start = df_ministers.groupby('memberID')['start_date'].apply(list).to_dict()#już pomijamy fakt ewentualnej przerwy w kadencji w wyliczeniach
for k in min_years_start.keys():
  min_years_start[k] = min(min_years_start[k])

min_years_end = df_ministers.groupby('memberID')['end_date'].apply(list).to_dict()#już pomijamy fakt ewentualnej przerwy w kadencji w wyliczeniach
for k in min_years_end.keys():
  min_years_end[k] = max(min_years_end[k])

min_years_end

In [ ]:
df_only_curr_minister = df.loc[(df['memberID'].isin(ministers_ids)) & (df['date'] < df['memberID'].apply(lambda x: min_years_end.get(x))) & (df['date'] > df['memberID'].apply(lambda x: min_years_start.get(x)))] #& (df['date'].dt> min_years_end[df['memberID']])

In [ ]:
lemmas_curr_min = df_only_curr_minister.speech_en.apply(lambda doc: [token.lemma_ for token in doc if not token.is_stop if not token.is_punct])

cloud_from_lemmas(lemmas_curr_min)

Taka analiza pozwala zauważyć, że słowa typowe dla polityki zmieniają się wraz z pozycją. Także używane zwroty grzecznościowe (z pewnością rozpoczynające część z wystąpień). Prócz słów typowych można doszukiwać się także innych, przykładowo symbol euro sugerujący podnoszenie kwestii wydatków lub planowania budżetu.

Teraz spróbujmy odflitrować niektóre słowa i powtórzyć tę samą chmurę.

In [ ]:
not_interesting = set(["department", "deputy", "provide", "service", "include", "project", "issue", "minister", "matter", "government", "statement", "year", "person", "people", "ask", "number", "detail", "supply"])


In [ ]:
lemmas_specific = df_only_curr_minister.speech_en.apply(lambda doc: [token.lemma_ for token in doc if not token.is_stop if not token.is_punct if not token.lemma_.lower() in not_interesting])

cloud_from_lemmas(lemmas_specific)

Teraz na lupę zbiór komplementarny do tego, czyli nieministrowie w trakcie wygłaszanie przemówień.

In [ ]:
lemmas_specific = df.loc[~df["speechID"].isin(set(df_only_curr_minister["speechID"]))].speech_en.apply(lambda doc: [token.lemma_ for token in doc if not token.is_stop if not token.is_punct if not token.lemma_.lower() in not_interesting])

cloud_from_lemmas(lemmas_specific)

Można zauważyć różnice w danych tematach. W pierwszym wypadku istotne wydają się np. work, school, Ireland, euro, drugi to np. time, County, child, Health, Bill.

## Barploty ministrowie vs. nieministrowie

In [ ]:
def plot_counts(counts):
  fig = px.bar(counts,orientation='h', y='word', x='count')

  fig['layout']['yaxis']['autorange'] = "reversed"
  fig.update_layout(bargap=0.30, font={'size':10})
  return fig

In [ ]:
#word count for certain group
lemmas_only_ministers = df_only_curr_minister.speech_en.apply(lambda doc: [token.lemma_ for token in doc if not token.is_stop if not token.is_punct])
word_counts_only_ministers = Counter(lemmas_only_ministers.sum())

lemmas_only_non_ministers = df.loc[~df["speechID"].isin(set(df_only_curr_minister["speechID"]))].speech_en.apply(lambda doc: [token.lemma_ for token in doc if not token.is_stop if not token.is_punct])
word_counts_only_non_ministers = Counter(lemmas_only_non_ministers.sum())



Sporządzimy wykres, gdzie będą znajdować się słowa najcześciej wypowiedziane przez daną grupę, nie występujące ani raz w drugiej grupie. Najpierw dla ministrów

Ministrowie:

In [ ]:
counts = pd.DataFrame(Counter({k: v for k, v in word_counts_only_ministers.items() if k not in word_counts_only_non_ministers.keys()}).most_common(30), columns=['word', 'count'])
plot_counts(counts)

In [ ]:
counts = pd.DataFrame(Counter({k: v for k, v in word_counts_only_non_ministers.items() if k not in word_counts_only_ministers.keys()}).most_common(30), columns=['word', 'count'])

plot_counts(counts)

Bardziej interesujący wydaje się wykres numer 2. Zestaw słow jak disgrace, scandal, shame, guillotine, outragous, coalition, jail, comfort, hang, fool, syndrome, savage. Są to słowa silnie nacechowane. Po pierwsze widzimy, że ministrowie mogą być o wiele bardziej zachowawczy w doborze słów. Po drugie takie słowa mogą służyć do formułowania zarzutów wobec rządzącej obecnie koalicji.

In [ ]:
non_min_speeches = df.loc[~df["speechID"].isin(set(df_only_curr_minister["speechID"]))]

In [ ]:
non_min_speeches.loc[list(lemmas_only_non_ministers.apply(lambda x: "jail" in x))]["speech"]

In [ ]:
non_min_speeches.loc[list(lemmas_only_non_ministers.apply(lambda x: "jail" in x))]["speech"][3743888]

Można sobie poczytać kilka wybranych. Pierwsze nie zawierają gróźb więzenia dla "niekompetentnych rządzących", jednak i tak dosyć łatwo znaleźć zarzuty wobec rządu. Minister for Justice and Law Reform często jest wspominany w tych przemówieniach

In [ ]:
non_min_speeches.loc[list(lemmas_only_non_ministers.apply(lambda x: "guillotine" in x))]["speech"]

In [ ]:
non_min_speeches.loc[list(lemmas_only_non_ministers.apply(lambda x: "guillotine" in x))]["speech"][3763831]

In [ ]:
non_min_speeches.loc[list(lemmas_only_non_ministers.apply(lambda x: "guillotine" in x))]["speech"][3750224]

Allocation of time or 'guillotine' motions have been used by governments to limit the amount of time that MPs can spend debating a particular stage of a Bill in the House of Commons.

Wyrażanie niezadowolenia wobec rządu.

Teraz już ostatnie shame. Do analizy pozostałych zachęcamy samodzielnie.

In [ ]:
non_min_speeches.loc[list(lemmas_only_non_ministers.apply(lambda x: "scandal" in x))]["speech"]

In [ ]:
non_min_speeches.loc[list(lemmas_only_non_ministers.apply(lambda x: "scandal" in x))]["speech"][3783355    ]

Pierwsze zdanie z powyższego outputu.

## TF-IDF

In [ ]:
# !pip install scikit-learn

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# lemmas_common = df.speech_en.apply(lambda doc: [token.lemma_ for token in doc if not token.is_stop if not token.is_punct])

# vectorizer = TfidfVectorizer()
# X = vectorizer.fit_transform(lemmas_common.apply(lambda x: " ".join(x)))
# vectorizer.get_feature_names_out()

# tfidf_df = pd.DataFrame(X.toarray(), index = lemmas_common.keys(),columns=vectorizer.get_feature_names())

# tfidf_df = tfidf_df.stack().reset_index()

# tfidf_df = tfidf_df.rename(columns={0:'tfidf','level_1': 'term', 'level_0': 'speech_id'})
# to_cos = tfidf_df.sort_values(by=['speech_id','tfidf'], ascending=[True,False]).groupby(['speech_id'])
# Counter(to_cos["term"]).most_common(30)

W końcu nie używamy go, był prototyp do wyciągnięcia np. 5 słow z najwyższym indexem z każdego speecha i potem grupowanie ich np. dla konkretnej partii i wyciąganie most common.

## Względem departamnetów

Ten notebook i tak jest długi prezentujemy tylko resort finansów, można tak zrobić pozostałe i powyciągać wnioski zminiając nazwę departamentu.

In [ ]:
lemmas_fin = df.loc[df['memberID'].isin(dict_depart["Finance"])].speech_en.apply(lambda doc: [token.lemma_ for token in doc if not token.is_stop if not token.is_punct if not token.lemma_.lower() in not_interesting])
word_counts_fin = Counter(lemmas_fin.sum())
counts_fin = pd.DataFrame(word_counts_fin.most_common(30), columns=['word', 'count'])
plot_counts(counts_fin)

Nie powinna dziwić np. popularność słowa tax czy euro w przemowach ministra finansów

# Institutional Grammar tagging algorithm

In [ ]:
"""
Program to implement and demonstrate institutional grammar tagging algorithm. Due to some problems with neuralcoref
library, coreference resolution does not work.
"""


# Library for coreference resolution
# !pip install neuralcoref

# Neuralcoref is not compatible with spaCy v3.0
# import neuralcoref

# neuralcoref.add_to_pipe(en)
from typing import Tuple, Union, List, Any, Optional

import spacy as spacy


def institutional_grammar_tagging_algorithm(sentence: spacy.tokens.Span) -> \
        Tuple[Union[List[Any], Any], Union[List[Any], Any], List[Optional[Any]]]:
    """
    Implementation of institutional grammar tagging algorithm.
    :param sentence: deontic sentence
    :return: attributes, objects and verbs of the given deontic sentence
    """
    attributes = []
    objects = []
    verbs = []
    verb = sentence.root

    while verb is not None:
        attr = verb
        verb = None
        verbs.append(attr)

        newSubj = [c for c in attr.children if c.dep_ == "nsubj"]
        newPassiveSubj = [c for c in attr.children if c.dep_ == "nsubjpass"]

        if len(newSubj) == 0 and len(newPassiveSubj) == 0:
            attributes = [clausal for c in attr.children for clausal in c.children if c.dep_ == "csubj"]

        attributes = attributes + newSubj
        objects = objects + newPassiveSubj + [c for c in attr.children if c.dep_ == "dobj"]

        if attr.dep_ == "conj" and attr.pos_ == "VERB":
            verb = attr.head

    for subject in attributes:
        for attr in attributes:
            if attr.dep_ == "conj":
                attributes.append(subject)

            attributes = attributes + [s for s in subject.children if s.dep_ == "conj"]

            if subject.pos_ == "PRONOUN":
                # coreference resolution does not work
                pass

    for objectx in objects:
        objects = objects + [s for s in objectx.children if s.dep_ == "conj"]

    return attributes, objects, verbs


In [ ]:
testcase1 = en("Designers, builders, and manufacturers should submit details and documentation to the assesment body. Any decisions made by a machine must be logged and retained.")
for sent in testcase1.sents:
  print(institutional_grammar_tagging_algorithm(sent))
print("\n")
for sent in testcase1.sents:
  print(sent.root)

In [ ]:
from spacy import displacy
for sent in testcase1.sents:
  displacy.render(sent, jupyter=True)

In [ ]:
spacy.explain("appos")